# ANALISE DO SPOTIFY

### Imports

In [1]:
import os
import json
import pandas as pd
from typing import Dict, Any

### Metodos

In [2]:
def folder_to_df_merged(folder_path: str) -> Dict[str, pd.DataFrame]:
    result = {}

    for file in os.listdir(folder_path):
        if not file.endswith(".json"):
            continue

        path = os.path.join(folder_path, file)

        with open(path, encoding="utf-8") as f:
            data: Any = json.load(f)

        if isinstance(data, list) and all(isinstance(i, dict) for i in data):
            df = pd.json_normalize(data)

        elif isinstance(data, dict):
            list_keys = [k for k, v in data.items() if isinstance(v, list)]

            if list_keys:
                main_key = list_keys[0]
                df = pd.json_normalize(
                    data,
                    record_path=main_key,
                    meta=[k for k in data.keys() if k != main_key],
                    errors="ignore"
                )
            else:
                df = pd.DataFrame([data])

        else:
            df = pd.DataFrame()

        result[file.replace(".json", "")] = df
    
    return pd.concat(result.values(), ignore_index=True)


def join_jsons_in_one_json(pasta, nome_arquivo_final="json_grandao.json"):
    json_final = []

    for arquivo in os.listdir(pasta):
        if arquivo.endswith(".json") and arquivo != nome_arquivo_final:
            caminho = os.path.join(pasta, arquivo)
            with open(caminho, "r", encoding="utf-8") as f:
                dados = json.load(f)
                json_final.extend(dados)

    caminho_saida = os.path.join(pasta, nome_arquivo_final)

    with open(caminho_saida, "w", encoding="utf-8") as f:
        json.dump(json_final, f, ensure_ascii=False, indent=2)

    return caminho_saida


def add_field_user_in_jsons(pasta, user, campo_user="user"):

    for arquivo in os.listdir(pasta):
        if not arquivo.endswith(".json"):
            continue

        caminho = os.path.join(pasta, arquivo)

        with open(caminho, "r", encoding="utf-8") as f:
            dados = json.load(f)

        if isinstance(dados, list):
            for doc in dados:
                if isinstance(doc, dict):
                    doc[campo_user] = user

        elif isinstance(dados, dict):
            dados[campo_user] = user

        else:
            continue

        with open(caminho, "w", encoding="utf-8") as f:
            json.dump(dados, f, ensure_ascii=False, indent=2)